<a href="https://colab.research.google.com/github/AMenuire/twitter/blob/master/FlightSearch_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
#@title Vols depuis CDG
import requests
import os
import json
import pandas as pd
import numpy as np
from IPython.display import Image, HTML
import datetime
from datetime import date
print(date.today())
 
origine_IATA = "CDG"
destination_IATA = "ABJ;BEY;BKO;BGF;BZV;CKY;COO;DLA;DSS;FIH;LBV;LFW;NDJ;NIM;NKC;NSI;TNR;CAY;FDF;PTP;RUN"  
destination_IATA = "ABJ;SYD" 
#date_from_included = str(date.today()+datetime.timedelta(days=0))
date_from_included = str(date.today() + datetime.timedelta( (0-date.today().weekday()) % 7 ))

date_until_included = str(date.today()+datetime.timedelta(days=10)) #"2020-11-02" 
 
destination = destination_IATA.replace(" ", "")
destination = destination.split(";")
 
key = 'mhud7t8j2x5bpprwawsurdq3'
secret = 'uHRXh55W5t'
 
airlineCode = []
flightNumber = []
aircraftType = []
flightScheduleDate = []
departureAirportCode = []
departureTimeScheduled = []
departureTimeLatestPublished = []
arrivalAirportCode = []
arrivalTimeScheduled = []
arrivalTimeLatestPublished = []
viaAirportCode = []
status = []
#irregularity = []
publicStatus = []
flightRoute = []
 
#-H "accept-language:en-GB" -H "Api-Key:<userkeytoreplace>""
 
#headers = {"Authorization": "Bearer {}".format(bearer_token)}
headers = {"Api-Key": key}
 
for dest in destination:
  url = "https://api.airfranceklm.com/opendata/flightstatus/?origin=" + origine_IATA + "&destination=" + dest + "&departureCity=&arrivalCity=&startRange=" + date_from_included + "T00:00:00Z&endRange=" + date_until_included + "T23:59:59Z&movementType=D&pageSize=500&pageNumber=0&carrierCode=&flightNumber=&operationalSuffix=&aircraftRegistration=&aircraftType=&timeType=&timeOriginType="
  response = requests.request("GET", url, headers=headers)
  if response.status_code != 200:
      print(origine_IATA + " to " + dest + ": ERROR")
      continue
      #raise Exception(response.status_code, response.text)
  json_response = response.json()
  operationalFlights = json_response['operationalFlights']
  for flight in operationalFlights:
    airlineCode.append(flight.get('airline').get('code'))
    flightNumber.append(flight.get('flightNumber'))
    aircraftType.append(flight.get('flightLegs')[0].get('aircraft').get('typeCode'))
    flightScheduleDate.append(flight.get('flightScheduleDate'))
    viaAirport = " "
    depTime = flight.get('flightLegs')[0].get('departureInformation').get('times').get('scheduled') #ne sera mis dans le tableau que si tous les legs sont annulés. dans ce cas on ne sait pas quelle leg prendre pour décider de l'heure de départ donc on prend la première.
    arrTime = flight.get('flightLegs')[0].get('arrivalInformation').get('times').get('scheduled') #idem commentaire ci-dessus
    for leg in flight.get('flightLegs'):
      if leg.get('irregularity').get('cancelled') == 'N':
        if leg.get('departureInformation').get('airport').get('code') == origine_IATA:
          depTime = leg.get('departureInformation').get('times').get('scheduled')
        if not leg.get('departureInformation').get('airport').get('code') == origine_IATA and leg.get('arrivalInformation').get('airport').get('code') == dest :
          viaAirport = leg.get('departureInformation').get('airport').get('code')
        if leg.get('arrivalInformation').get('airport').get('code') == dest:
          arrTime = leg.get('arrivalInformation').get('times').get('scheduled')        
    departureAirportCode.append(origine_IATA)
    departureTimeScheduled.append(depTime)
    departureTimeLatestPublished.append(flight.get('flightLegs')[0].get('departureInformation').get('times').get('latestPublished'))
    arrivalAirportCode.append(dest)
    arrivalTimeScheduled.append(arrTime)
    arrivalTimeLatestPublished.append(flight.get('flightLegs')[0].get('arrivalInformation').get('times').get('latestPublished'))
    viaAirportCode.append(viaAirport)
    status.append(flight.get('flightLegs')[0].get('status'))
    #irregularity.append(flight.get('flightLegs')[0].get('irregularity').get('cancelled'))
    publicStatus.append(flight.get('flightStatusPublic'))
    flightRoute.append('-'.join(flight.get('route')))
 
flights = pd.DataFrame()
flights['airlineCode'] = airlineCode
flights['flightNumber'] = flightNumber
flights['aircraftType'] = aircraftType
flights['flightScheduleDate'] = flightScheduleDate
flights['departureAirportCode'] = departureAirportCode
flights['arrivalAirportCode'] = arrivalAirportCode
flights['viaAirportCode'] = viaAirportCode
flights['departureTimeScheduled'] = departureTimeScheduled
flights['departureTimeLatestPublished'] = departureTimeLatestPublished
 
flights['arrivalTimeScheduled'] = arrivalTimeScheduled
flights['arrivalTimeLatestPublished'] = arrivalTimeLatestPublished
flights['status'] = status
#flights['irregularity'] = irregularity
flights['publicStatus'] = publicStatus
flights['flightRoute'] = flightRoute
flights['departureTimeScheduled'] = pd.to_datetime(flights['departureTimeScheduled'])
flights['arrivalTimeScheduled'] = pd.to_datetime(flights['arrivalTimeScheduled'])
 
def miseenforme(row):
    url = "https://wwws.airfrance.fr/flight-status/flight-list?date=" + str(row.flightScheduleDate)[0:10] + "&originAirportCode=" + str(row.departureAirportCode) + "&destinationAirportCode=" + str(row.arrivalAirportCode)
    row.summary = '<b> D%s A%s: </b> <a target="_blank" href="%s">%s</a>' % (str(row.departureTimeScheduled.time())[0:5],str(row.arrivalTimeScheduled.time())[0:5],url,str(row.airlineCode)+str(row.flightNumber))
    #if not row.flightRoute == str(row.departureAirportCode) + "-" + str(row.arrivalAirportCode):
    route = row.flightRoute
    row.summary = row.summary + " (" + route + ")" 
    if not str(row.airlineCode) == 'AF':
      row.summary = ""
    if row.publicStatus=='CANCELLED':
      row.summary = "<s>" + row.summary + "</s>"
    if str(row.arrivalAirportCode) not in str(row.flightRoute):
      row.summary = ""
    return row.summary
 
def miseEnFormeJour(row):
  weekday = row.flightScheduleDate.weekday()
  if weekday == 0:
    jour = '(lundi)'
  if weekday == 1:
    jour = '(mardi)'
  if weekday == 2:
    jour = '(mercredi)'
  if weekday == 3:
    jour = '(jeudi)'
  if weekday == 4:
    jour = '(vendredi)'
  if weekday == 5:
    jour = '(samedi)'
  if weekday == 6:
    jour = '(dimanche)'
  row.flightScheduleDay = jour 
  return row.flightScheduleDay
 
flights['flightScheduleDate'] = pd.to_datetime(flights['flightScheduleDate'])
flights['flightScheduleDate'] = flights.apply(lambda row: str(row.flightScheduleDate)[0:10] + ' ' + miseEnFormeJour(row), axis=1)
#
#flights['summary'] = flights.apply(lambda row: ('<b> %s </b>(%s)' % (str(row.airlineCode)+str(row.flightNumber),str(row.departureTimeScheduled.time())[0:5])).format(), axis=1)
flights['summary'] = flights.apply(lambda row: miseenforme(row), axis=1)
flights2 = flights.groupby(['flightScheduleDate','arrivalAirportCode'])
pd.set_option('display.max_colwidth', None)  
flights2 = flights2['summary'].apply(' <br> '.join).reset_index()
#flights2['arrivalAirportCode'] = flights2['arrivalAirportCode'].apply('<b>{}</b>?'.format)
#flights2['arrivalAirportCode'] = [f'<b>{x}</b>' for x in flights2['arrivalAirportCode']]
flights2 = flights2.pivot(index='arrivalAirportCode', columns='flightScheduleDate', values='summary')
flights2 = flights2.replace(np.nan, '', regex=True)
flights2 = flights2.reindex(destination) #réordonne le df selon la liste des destinations (telle que l'utilisateur l'a entrée)
a = HTML(flights2.to_html(escape=False))
a

2021-09-01
CDG to SYD: ERROR


flightScheduleDate,2021-09-06 (lundi),2021-09-07 (mardi),2021-09-08 (mercredi),2021-09-09 (jeudi),2021-09-10 (vendredi),2021-09-11 (samedi)
arrivalAirportCode,,,,,,
ABJ,D14:25 A18:45: AF702 (CDG-ABJ) D18:00 A22:30: AF704 (CDG-ABJ),D14:25 A18:45: AF702 (CDG-ABJ) D18:00 A22:30: AF704 (CDG-ABJ),D14:25 A18:45: AF702 (CDG-ABJ) D18:00 A22:30: AF704 (CDG-ABJ),D14:25 A18:45: AF702 (CDG-ABJ) D18:00 A22:30: AF704 (CDG-ABJ),D14:25 A18:45: AF702 (CDG-ABJ) D18:00 A22:30: AF704 (CDG-ABJ),D14:25 A18:45: AF702 (CDG-ABJ) D18:00 A22:30: AF704 (CDG-ABJ)
SYD,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
flights2.reindex(destination)

flightScheduleDate,2021-01-18 (lundi),2021-01-19 (mardi),2021-01-20 (mercredi),2021-01-21 (jeudi),2021-01-22 (vendredi),2021-01-23 (samedi),2021-01-24 (dimanche),2021-01-25 (lundi)
arrivalAirportCode,,,,,,,,
ABJ,<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ),<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ),<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ),<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ),<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ),<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ),<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ),<br> <b> D14:05 A19:30: </b> AF702 (CDG-ABJ)
BKO,<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB),<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB),<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB),<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB),<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB),<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB),<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB),<b> D10:10 A14:55: </b> AF530 (CDG-BKO-ROB)
BGF,,<b> D08:35 A15:15: </b> AF775 (CDG-BGF-NSI),,,<s><b> D08:55 A15:40: </b> AF775 (CDG-BGF-NSI-CDG)</s>,<s><b> D08:35 A15:25: </b> AF878 (CDG-BGF-NDJ)</s>,<b> D08:35 A15:25: </b> AF878 (CDG-BGF-NDJ),
BZV,<b> D10:15 A18:10: </b> AF716 (CDG-BZV-FIH-CDG) <br> <b> D10:15 A20:40: </b> AF756 (CDG-FIH-BZV-CDG),<b> D10:15 A20:40: </b> AF722 (CDG-FIH-BZV-CDG),<b> D10:15 A18:10: </b> AF716 (CDG-BZV-FIH-CDG),<b> D10:15 A20:40: </b> AF722 (CDG-FIH-BZV-CDG),<b> D10:15 A18:10: </b> AF716 (CDG-BZV-FIH-CDG) <br> <b> D10:15 A20:40: </b> AF736 (CDG-FIH-BZV-CDG),<b> D09:05 A17:00: </b> AF722 (CDG-BZV-FIH-CDG),<b> D09:05 A17:00: </b> AF736 (CDG-BZV-FIH-CDG) <br> <b> D10:15 A18:10: </b> AF716 (CDG-BZV-FIH-CDG),<b> D10:15 A18:10: </b> AF716 (CDG-BZV-FIH-CDG) <br> <b> D10:15 A20:40: </b> AF756 (CDG-FIH-BZV-CDG)
CKY,<b> D12:45 A20:45: </b> AF727 (CDG-NKC-CKY-CDG),<b> D12:30 A20:30: </b> AF596 (CDG-FNA-CKY-CDG),<b> D13:20 A18:45: </b> AF724 (CDG-CKY-NKC-CDG),<b> D12:30 A20:30: </b> AF596 (CDG-FNA-CKY-CDG),<b> D13:20 A18:45: </b> AF724 (CDG-CKY-NKC-CDG),<b> D12:30 A20:30: </b> AF596 (CDG-FNA-CKY-CDG),<b> D12:45 A20:45: </b> AF751 (CDG-NKC-CKY-CDG),<b> D12:45 A20:45: </b> AF727 (CDG-NKC-CKY-CDG)
COO,<b> D14:35 A21:00: </b> AF804 (CDG-COO),,<b> D14:35 A21:00: </b> AF804 (CDG-COO),,<b> D14:35 A21:00: </b> AF804 (CDG-COO),,<b> D14:35 A21:00: </b> AF804 (CDG-COO),<b> D14:35 A21:00: </b> AF804 (CDG-COO)
DLA,<br> <b> D12:00 A18:35: </b> AF983 (CDG-DLA-NSI-CDG),,<s><b> D10:30 A17:15: </b> AF958 (CDG-DLA-SSG)</s>,<b> D12:55 A21:55: </b> AF982 (CDG-NSI-DLA-CDG),<s><b> D10:30 A17:15: </b> AF958 (CDG-DLA-SSG)</s> <br> <b> D12:00 A18:35: </b> AF983 (CDG-DLA-NSI-CDG),<s><b> D10:30 A17:15: </b> AF958 (CDG-DLA-SSG)</s>,<b> D12:55 A21:55: </b> AF982 (CDG-NSI-DLA-CDG),<br> <b> D12:00 A18:35: </b> AF983 (CDG-DLA-NSI-CDG)
DSS,<b> D15:40 A20:35: </b> AF718 (CDG-DSS),<b> D15:40 A20:35: </b> AF718 (CDG-DSS),<b> D15:40 A20:35: </b> AF718 (CDG-DSS),<b> D15:40 A20:35: </b> AF718 (CDG-DSS),<b> D15:40 A20:35: </b> AF718 (CDG-DSS),<b> D15:40 A20:35: </b> AF718 (CDG-DSS),<b> D15:40 A20:35: </b> AF718 (CDG-DSS),<b> D15:40 A20:35: </b> AF718 (CDG-DSS)
EZE,<s><b> D23:40 A09:20: </b> AF228 (CDG-EZE)</s>,<s><b> D23:40 A09:20: </b> AF228 (CDG-EZE)</s>,<s></s> <br> <b> D23:50 A09:30: </b> AF228 (CDG-EZE),<s><b> D23:40 A09:20: </b> AF228 (CDG-EZE)</s>,<b> D21:50 A07:30: </b> AF228 (CDG-EZE),,<b> D23:20 A09:00: </b> AF228 (CDG-EZE),<s><b> D23:40 A09:20: </b> AF228 (CDG-EZE)</s>


In [ ]:
#@title Vols depuis ORY
import requests
import os
import json
import pandas as pd
import numpy as np
from IPython.display import Image, HTML
import datetime
from datetime import date
print(date.today())
 
origine_IATA = "ORY"
destination_IATA = "FDF;PTP;CAY;RUN" 
date_from_included = str(date.today()+datetime.timedelta(days=5))
date_until_included = str(date.today()+datetime.timedelta(days=10)) #"2020-11-02" 
 
destination = destination_IATA.replace(" ", "")
destination = destination.split(";")
 
key = 'mhud7t8j2x5bpprwawsurdq3'
secret = 'uHRXh55W5t'
 
airlineCode = []
flightNumber = []
aircraftType = []
flightScheduleDate = []
departureAirportCode = []
departureTimeScheduled = []
departureTimeLatestPublished = []
arrivalAirportCode = []
arrivalTimeScheduled = []
arrivalTimeLatestPublished = []
viaAirportCode = []
status = []
#irregularity = []
publicStatus = []
flightRoute = []
 
#-H "accept-language:en-GB" -H "Api-Key:<userkeytoreplace>""
 
#headers = {"Authorization": "Bearer {}".format(bearer_token)}
headers = {"Api-Key": key}
 
for dest in destination:
  url = "https://api.airfranceklm.com/opendata/flightstatus/?origin=" + origine_IATA + "&destination=" + dest + "&departureCity=&arrivalCity=&startRange=" + date_from_included + "T00:00:00Z&endRange=" + date_until_included + "T23:59:59Z&movementType=D&pageSize=500&pageNumber=0&carrierCode=&flightNumber=&operationalSuffix=&aircraftRegistration=&aircraftType=&timeType=&timeOriginType="
  response = requests.request("GET", url, headers=headers)
  if response.status_code != 200:
      print(origine_IATA + " to " + dest + ": ERROR")
      continue
      #raise Exception(response.status_code, response.text)
  json_response = response.json()
  operationalFlights = json_response['operationalFlights']
  for flight in operationalFlights:
    airlineCode.append(flight.get('airline').get('code'))
    flightNumber.append(flight.get('flightNumber'))
    aircraftType.append(flight.get('flightLegs')[0].get('aircraft').get('typeCode'))
    flightScheduleDate.append(flight.get('flightScheduleDate'))
    viaAirport = " "
    depTime = flight.get('flightLegs')[0].get('departureInformation').get('times').get('scheduled') #ne sera mis dans le tableau que si tous les legs sont annulés. dans ce cas on ne sait pas quelle leg prendre pour décider de l'heure de départ donc on prend la première.
    arrTime = flight.get('flightLegs')[0].get('arrivalInformation').get('times').get('scheduled') #idem commentaire ci-dessus
    for leg in flight.get('flightLegs'):
      if leg.get('irregularity').get('cancelled') == 'N':
        if leg.get('departureInformation').get('airport').get('code') == origine_IATA:
          depTime = leg.get('departureInformation').get('times').get('scheduled')
        if not leg.get('departureInformation').get('airport').get('code') == origine_IATA and leg.get('arrivalInformation').get('airport').get('code') == dest :
          viaAirport = leg.get('departureInformation').get('airport').get('code')
        if leg.get('arrivalInformation').get('airport').get('code') == dest:
          arrTime = leg.get('arrivalInformation').get('times').get('scheduled')        
    departureAirportCode.append(origine_IATA)
    departureTimeScheduled.append(depTime)
    departureTimeLatestPublished.append(flight.get('flightLegs')[0].get('departureInformation').get('times').get('latestPublished'))
    arrivalAirportCode.append(dest)
    arrivalTimeScheduled.append(arrTime)
    arrivalTimeLatestPublished.append(flight.get('flightLegs')[0].get('arrivalInformation').get('times').get('latestPublished'))
    viaAirportCode.append(viaAirport)
    status.append(flight.get('flightLegs')[0].get('status'))
    #irregularity.append(flight.get('flightLegs')[0].get('irregularity').get('cancelled'))
    publicStatus.append(flight.get('flightStatusPublic'))
    flightRoute.append('-'.join(flight.get('route')))
 
flights = pd.DataFrame()
flights['airlineCode'] = airlineCode
flights['flightNumber'] = flightNumber
flights['aircraftType'] = aircraftType
flights['flightScheduleDate'] = flightScheduleDate
flights['departureAirportCode'] = departureAirportCode
flights['arrivalAirportCode'] = arrivalAirportCode
flights['viaAirportCode'] = viaAirportCode
flights['departureTimeScheduled'] = departureTimeScheduled
flights['departureTimeLatestPublished'] = departureTimeLatestPublished
 
flights['arrivalTimeScheduled'] = arrivalTimeScheduled
flights['arrivalTimeLatestPublished'] = arrivalTimeLatestPublished
flights['status'] = status
#flights['irregularity'] = irregularity
flights['publicStatus'] = publicStatus
flights['flightRoute'] = flightRoute
flights['departureTimeScheduled'] = pd.to_datetime(flights['departureTimeScheduled'])
flights['arrivalTimeScheduled'] = pd.to_datetime(flights['arrivalTimeScheduled'])
 
def miseenforme(row):
    row.summary = '<b> D%s A%s: </b> %s' % (str(row.departureTimeScheduled.time())[0:5],str(row.arrivalTimeScheduled.time())[0:5],str(row.airlineCode)+str(row.flightNumber))
    #if not row.flightRoute == str(row.departureAirportCode) + "-" + str(row.arrivalAirportCode):
    route = row.flightRoute
    row.summary = row.summary + " (" + route + ")" 
    if not str(row.airlineCode) == 'AF':
      row.summary = ""
    if row.publicStatus=='CANCELLED':
      row.summary = "<s>" + row.summary + "</s>"
    if str(row.arrivalAirportCode) not in str(row.flightRoute):
      row.summary = ""
    return row.summary
 
def miseEnFormeJour(row):
  weekday = row.flightScheduleDate.weekday()
  if weekday == 0:
    jour = '(lundi)'
  if weekday == 1:
    jour = '(mardi)'
  if weekday == 2:
    jour = '(mercredi)'
  if weekday == 3:
    jour = '(jeudi)'
  if weekday == 4:
    jour = '(vendredi)'
  if weekday == 5:
    jour = '(samedi)'
  if weekday == 6:
    jour = '(dimanche)'
  row.flightScheduleDay = jour 
  return row.flightScheduleDay
 
flights['flightScheduleDate'] = pd.to_datetime(flights['flightScheduleDate'])
flights['flightScheduleDate'] = flights.apply(lambda row: str(row.flightScheduleDate)[0:10] + ' ' + miseEnFormeJour(row), axis=1)
#
#flights['summary'] = flights.apply(lambda row: ('<b> %s </b>(%s)' % (str(row.airlineCode)+str(row.flightNumber),str(row.departureTimeScheduled.time())[0:5])).format(), axis=1)
flights['summary'] = flights.apply(lambda row: miseenforme(row), axis=1)
flights2 = flights.groupby(['flightScheduleDate','arrivalAirportCode'])
pd.set_option('display.max_colwidth', None)  
flights2 = flights2['summary'].apply(' <br> '.join).reset_index()
#flights2['arrivalAirportCode'] = flights2['arrivalAirportCode'].apply('<b>{}</b>?'.format)
#flights2['arrivalAirportCode'] = [f'<b>{x}</b>' for x in flights2['arrivalAirportCode']]
flights2 = flights2.pivot(index='arrivalAirportCode', columns='flightScheduleDate', values='summary')
flights2 = flights2.replace(np.nan, '', regex=True)
a = HTML(flights2.to_html(escape=False))
a

2021-01-13


flightScheduleDate,2021-01-18 (lundi),2021-01-19 (mardi),2021-01-20 (mercredi),2021-01-21 (jeudi),2021-01-22 (vendredi),2021-01-23 (samedi)
arrivalAirportCode,,,,,,
CAY,D10:45 A15:55: AF852 (ORY-CAY),D10:45 A15:55: AF852 (ORY-CAY),D10:45 A15:55: AF852 (ORY-CAY),D10:45 A15:55: AF852 (ORY-CAY),D10:45 A15:55: AF852 (ORY-CAY),D10:45 A15:55: AF852 (ORY-CAY)
FDF,D11:45 A15:35: AF842 (ORY-FDF) D14:20 A18:10: AF848 (ORY-FDF),D11:45 A15:35: AF842 (ORY-FDF) D14:20 A18:10: AF848 (ORY-FDF),D11:45 A15:35: AF842 (ORY-FDF) D14:20 A18:10: AF848 (ORY-FDF),D11:45 A15:35: AF842 (ORY-FDF) D14:20 A18:10: AF848 (ORY-FDF) D15:20 A18:45: AF810 (CDG-FDF),D11:45 A15:35: AF842 (ORY-FDF) D14:20 A18:10: AF848 (ORY-FDF),D11:45 A15:35: AF842 (ORY-FDF) D14:20 A18:10: AF848 (ORY-FDF) D15:20 A18:45: AF810 (CDG-FDF)
PTP,D11:15 A14:55: AF792 (ORY-PTP) D14:50 A18:45: AF766 (ORY-PTP) D15:50 A19:25: AF740 (CDG-PTP),D11:15 A14:55: AF792 (ORY-PTP) D14:50 A18:45: AF766 (ORY-PTP),D11:15 A14:55: AF792 (ORY-PTP) D14:50 A18:45: AF766 (ORY-PTP),D11:15 A14:55: AF792 (ORY-PTP) D14:50 A18:45: AF766 (ORY-PTP),D11:15 A14:55: AF792 (ORY-PTP) D14:50 A18:45: AF766 (ORY-PTP) D15:50 A19:25: AF740 (CDG-PTP),D11:15 A14:55: AF792 (ORY-PTP) D14:50 A18:45: AF766 (ORY-PTP)
RUN,D18:00 A08:00: AF644 (ORY-RUN) D20:45 A10:40: AF642 (ORY-RUN),D18:00 A08:00: AF644 (ORY-RUN) D20:45 A10:40: AF642 (ORY-RUN),D18:00 A08:00: AF644 (ORY-RUN) D20:45 A10:40: AF642 (ORY-RUN),D18:00 A08:00: AF644 (ORY-RUN) D20:45 A10:40: AF642 (ORY-RUN),D18:00 A08:00: AF644 (ORY-RUN) D20:45 A10:40: AF642 (ORY-RUN),D18:00 A08:00: AF644 (ORY-RUN) D20:45 A10:40: AF642 (ORY-RUN)


In [ ]:
#@title Exporter vers google drive
#from google.colab import drive
#from datetime import datetime as dt
#drive.mount('/gdrive')
file_name_optional = 'export27112020_1' #@param {type: "string"}
if (file_name_optional != ''):
  #path = "/gdrive/My Drive/data/timetables/" + file_name_optional + '.csv'
  path = "/content/drive/My Drive/data/timetables/" + file_name_optional + '.csv'
  flights2.to_csv(path,index = False,encoding='utf-8')
else:
  #path = "/gdrive/My Drive/data/timetables/" + str(dt.now()) + '.csv'
  path = "/content/drive/My Drive/data/timetables/" + str(dt.now()) + '.csv'
  flights2.to_csv(path,index = False,encoding='utf-8')
print ("Saved to "+path)

Saved to /content/drive/My Drive/data/timetables/export27112020_1.csv


In [ ]:
#@title Import from google drive
archive_csv = "/content/drive/My Drive/data/timetables/export05112020_2.csv"  #@param {type: "string"}
 
#@title Load .csv file
flights3 = pd.read_csv(archive_csv)
flights3 = flights3.replace(np.nan, '', regex=True)
print('Loaded')

Loaded


In [ ]:
#@title Comparer avec archive précédente

In [ ]:
#@title Comparer deux archives (methode 1)

archive1_csv = "/content/drive/My Drive/data/timetables/export16112020_1.csv"  #@param {type: "string"}
archive2_csv = "/content/drive/My Drive/data/timetables/export17112020_1.csv"  #@param {type: "string"}
archive1 = pd.read_csv(archive1_csv)
archive1 = archive1.replace(np.nan, '', regex=True)
archive2 = pd.read_csv(archive2_csv)
archive2 = archive2.replace(np.nan, '', regex=True)
archive1 = archive1.drop(archive1.columns[0], axis='columns')
archive2 = archive2.drop(archive2.columns[10], axis='columns')
pd.concat([archive1,archive2]).drop_duplicates(keep=False)
#flights2.reset_index()
#a = HTML(flights3.to_html(escape=False))
#a

,2020-11-17 (mardi),2020-11-18 (mercredi),2020-11-19 (jeudi),2020-11-20 (vendredi),2020-11-21 (samedi),2020-11-22 (dimanche),2020-11-23 (lundi),2020-11-24 (mardi),2020-11-25 (mercredi),2020-11-26 (jeudi)
10,,,,,,,,,,


In [ ]:
#@title Comparer deux archives (methode 2)
archive1_csv = "/content/drive/My Drive/data/timetables/export13112020_1.csv"  #@param {type: "string"}
archive2_csv = "/content/drive/My Drive/data/timetables/export14112020_1.csv"  #@param {type: "string"}
archive1 = pd.read_csv(archive1_csv)
archive1 = archive1.replace(np.nan, '', regex=True)
archive2 = pd.read_csv(archive2_csv)
archive2 = archive2.replace(np.nan, '', regex=True)
archive1 = archive1.drop(archive1.columns[0], axis='columns')
#archive1 = archive1.drop(10,0,inplace=False)
#archive1 = archive1.reset_index()
#archive1 = archive1.drop(archive1.columns[0], axis='columns')
archive2 = archive2.drop(archive2.columns[10], axis='columns')
archive1.compare(archive2, keep_shape=True)

2020-11-14 (samedi)        ... 2020-11-23 (lundi)      
                  self other  ...               self other
0                  NaN   NaN  ...                NaN   NaN
1                  NaN   NaN  ...                NaN   NaN
2                  NaN   NaN  ...                NaN   NaN
3                  NaN   NaN  ...                NaN   NaN
4                  NaN   NaN  ...                NaN   NaN
5                  NaN   NaN  ...                NaN   NaN
6                  NaN   NaN  ...                NaN   NaN
7                  NaN   NaN  ...                NaN   NaN
8                  NaN   NaN  ...                NaN   NaN
9                  NaN   NaN  ...                NaN   NaN
10                 NaN   NaN  ...                NaN   NaN
11                 NaN   NaN  ...                NaN   NaN
12                 NaN   NaN  ...                NaN   NaN
13                 NaN   NaN  ...                NaN   NaN
14                 NaN   NaN  ...                NaN   NaN
15                 NaN   NaN  ...                NaN   NaN
16                 NaN   NaN  ...                NaN   NaN
17                 NaN   NaN  ...                NaN   NaN
18                 NaN   NaN  ...                NaN   NaN

[19 rows x 20 columns]

In [ ]:
 
def miseenforme2(row):
    row.summary = 'D%s A%s: %s' % (str(row.departureTimeScheduled.time())[0:5],str(row.arrivalTimeScheduled.time())[0:5],str(row.airlineCode)+str(row.flightNumber))
    #if not row.flightRoute == str(row.departureAirportCode) + "-" + str(row.arrivalAirportCode):
    route = row.flightRoute
    row.summary = row.summary + " (" + route + ")" 
    if not str(row.airlineCode) == 'AF':
      row.summary = ""
    if row.publicStatus=='CANCELLED':
      row.summary = ""
    if str(row.arrivalAirportCode) not in str(row.flightRoute):
      row.summary = ""
    return row.summary
 

#flights['summary'] = flights.apply(lambda row: ('<b> %s </b>(%s)' % (str(row.airlineCode)+str(row.flightNumber),str(row.departureTimeScheduled.time())[0:5])).format(), axis=1)
flights['summary2'] = flights.apply(lambda row: miseenforme2(row), axis=1)
flights3 = flights.groupby(['flightScheduleDate','arrivalAirportCode'])
pd.set_option('display.max_colwidth', None)  
flights3 = flights3['summary2'].apply('\n'.join).reset_index()
flights3 = flights3.pivot(index='arrivalAirportCode', columns='flightScheduleDate', values='summary2')
flights3 = flights3.replace(np.nan, '', regex=True)
flights3.to_excel("output_.xlsx")